# NoSQL: Basic queries on non-relational databases

## NoSQL: Review

A NoSQL database is designed to handle data that are not structured in tabular relations. The data can be stored in key-value pairs (like Python dictionaries), documents (JSON), or graphs.

<img src="../assets/nosql_vs_sql.jpeg"  width = 500 px></img>

Unlike relational databases (SQL), NoSQL does not (necessarily) employ tables and rows. Also they are much more flexible since the structure of the entries is not predefined. Therefore you can add data without defining anything (even not the table itself).

[This article](https://www.integrate.io/blog/the-sql-vs-nosql-difference/) gives a recap of the biggest differences between SQL and NoSQL database systems:
|SQL|NoSQL|
|---|---|
|SQL databases are relational| NoSQL databases are non-relational. In NoSQL you cannot make relationships between tables (`JOIN`)|
|SQL databases are table-based |NoSQL databases are document, key-value, graph, or wide-column stores.|
|SQL databases use structured query language and have a predefined schema.| NoSQL databases have dynamic schemas for unstructured data.|
|SQL databases are vertically scalable (by adding processing power)|NoSQL databases are horizontally scalable (by adding servers/machines).|
|SQL databases are better for multi-row transactions|NoSQL is better for unstructured data like documents or JSON.|

In the image, below you will find examples of popular SQL and NoSQL databases. 

<img src="../assets/popular_examples_nosql_sql.jpeg" width =500px></img>

In the next section, we'll cover an example using MongoDB. You will see that we can translate many of the basic SQL queries you know.

## MongoDB

<img src="https://upload.wikimedia.org/wikipedia/fr/thumb/4/45/MongoDB-Logo.svg/527px-MongoDB-Logo.svg.png" />

A lot of companies provide NoSQL architectures. One of the most popular is [MongoDB](https://www.mongodb.com/). A MongoDB database contains collections (tables) of documents (entries). The documents are stored in the JSON format which is very convenient to handle with Python!

The syntax of the queries is based on JavaScript. They basically look like python dictionaries.

Before diving in some exercises, you can have a look to [this quick intro](https://www.mongodb.com/docs/manual/tutorial/query-documents/) about MongoDB queries.

### Creating the database

We have created and filled a MongoDB database for you. You probably know already the database. It is the list of country leaders that you have used already in the Wikipedia project.

You can set it up by deploying the Docker image we have pre-built:

In [ ]:
# You can run it from this notebook with:
!docker-compose up -d

# Or in your terminal with:
# docker-compose up -d

### Conneting to the database

For using MongoDB through Python you will need to install the `pymongo` library.

In [ ]:
from pymongo import MongoClient

# Creation of a MongoDB Client (by giving the host and the port)
client = MongoClient(host="localhost", port=27017)

# Instantiation of the database
db = client["becode"]

# Let's see which collections are in the database
db.list_collection_names()

### Basic Queries

**1. Show the first leader of the `leaders` collection by using the `find_one` method**

The corresponding `SQL` query would be:

```sql
SELECT * FROM leaders LIMIT 1;
```

In [ ]:
db["leaders"].find_one()

**2. Show the first leader of the collection whose country is Belgium**

To that purpose we will use the query as first parameter of the `find_one` function. The query is formatted as a dictionary.

In SQL it would be like adding a `WHERE` condition to the query:

```sql
SELECT * FROM leaders WHERE country = 'be' LIMIT 1;
```

In [ ]:
db["leaders"].find_one({"country": "be"})

**3. Select some fields to display**

Let's use the same query by displaying only the `first_name` and the `last_name` of the leader. It corresponds to a `SELECT` in SQL.

We will use the [project](https://www.mongodb.com/docs/manual/tutorial/project-fields-from-query-results/) as the second parameter of the function. It is also formatted as a dictionary whose the key contains the targeted field and the value is `1` if we want the field to be displayed

The corresponding SQL query would be:

```sql
SELECT first_name, last_name FROM leaders WHERE country = 'be' LIMIT 1;
```

In [ ]:
db["leaders"].find_one({"country": "be"}, {"first_name": 1, "last_name": 1})

We can also decide to not display a field. In that case we put `0` as value for the dictionary.

In [ ]:
db["leaders"].find_one({"country": "be"}, {"wikipedia_url": 0, "id": 0})

**4. Find the distinct countries**

The SQL equivalent:

```sql
SELECT DISTINCT country FROM leaders;
```

In [ ]:
db["leaders"].find().distinct("country")

**5. Find all the leaders who are still alive**

We can assume that they have no `death_date`, isn't?

In [ ]:
db["leaders"].find({"death_date": None}, {"last_name":1, "first_name":1, "country":1})

As you see the `find` command returns a cursor. No worries we can process it by using a simple Python loop!

In [ ]:
for leader in db["leaders"].find({"death_date": None}, {"last_name":1, "first_name":1, "country":1}):
    print(f"{leader['first_name']} {leader['last_name']} ({leader['country']})")

**6. Let's now insert the leader of tomorrow: you?**

As you know, MongoDB is flexible. It means that you can add entries although some fields are missing. Let's give a try:

In [ ]:
you = {
    'first_name': 'Laura',
    'last_name': 'Kriszgruber',
    'birth_date': '06/04/1997',
    'country': 'be'
}
db["leaders"].insert_one(you)

Let's have a look to your data!

In [ ]:
db["leaders"].find_one({"first_name": "Laura"})

We can observe two things here:
- A field `_id` has been automatically added by Mongo. This field is incremental. That means that it will always be higher than the previous element of the collection.
- Some fields are missing (the `place_of_birth` for instance). This is a property of NoSQL. All fields are not mandatory!

We can check missing values by using the query `{"$exists":False}`:

In [ ]:
for leader in db["leaders"].find({"place_of_birth":{"$exists":False}}):
    print(leader)

### Update data

Since your place of birth is missing in the data, let's add it now. The update function has two main arguments:
- a query that will select the entries to update
- an update operation. As always, it is formatted as a dictionary

In [ ]:
db["leaders"].update_one({"first_name": "Laura"}, {"$set": {"place_of_birth": "Seraing"}})

#### Remove data

Your dream is over ;-) Since you will not be a leader of tomorrow, we will remove you from the collection. The `delete` (or `delete_one`) function has one main argument: the query that will select the entries to be removed.

In [ ]:
db["leaders"].delete_one({"first_name": "Laura"})

## Your Turn!

Based on your knowledge and some Google search try to create the following queries:

- Remove the leaders who have an empty or null (`None`) last name
- Display all unique first names from the collection
- Transform all the dates of the dataset by a datetime object (they are currently strings which is not a good practice). You can use a python script that interacts with the DB instead of doing everything in a single query
- Display the 10 older leaders ordered by their birth date (search for how to sort and to use limits in MongoDB)
- Create a Python script that computes the numbers of leaders by country
- Do the same by using a MongoDB [aggregation pipeline](https://www.mongodb.com/docs/manual/aggregation/)

In [ ]:
# - Remove the leaders who have an empty or null (`None`) last name
result = db["leaders"].delete_many({
    "$or": [
        {"last_name": "None"},
        {"last_name": ""}
    ]
})
deleted_count = result.deleted_count
print(f"Number of documents deleted: {deleted_count}")

In [ ]:
# - Display all unique first names from the collection

db["leaders"].find().distinct("first_name")

In [ ]:
# - Transform all the dates of the dataset by a datetime object (they are currently strings which is not a good practice). 
# You can use a python script that interacts with the DB instead of doing everything in a single query
# Define the fields that need to be converted to datetime
import datetime


# Define the fields that need to be converted to datetime
date_fields = ["birth_date", "death_date", "start_mandate", "end_mandate"]

# Iterate over each document in the collection
for leader in db["leaders"].find():
    modified = False  # Flag to track if the document is modified

    for field in date_fields:
        if field in leader and leader[field]:
            if isinstance(leader[field], str):  # Only convert if it's a string
                try:
                    # Convert the date string to a datetime object
                    leader[field] = datetime.datetime.strptime(leader[field], "%Y-%m-%d")
                    modified = True  # Mark document as modified
                except ValueError:
                    print(f"Skipping invalid date format in document with ID: {leader['_id']} for field: {field}")

    # If the document was modified, update it in the database
    if modified:
        db["leaders"].replace_one({"_id": leader["_id"]}, leader)

print("Date fields have been successfully converted to datetime objects.")


In [ ]:
# - Display the 10 older leaders ordered by their birth date (search for how to sort and to use limits in MongoDB)

# Sort the leaders by birth date in ascending order
sorted_leaders = db["leaders"].find().sort("birth_date", 1).limit(10)

# Display the 10 oldest leaders
for leader in sorted_leaders:
    print(leader)

In [ ]:
# - Create a Python script that computes the numbers of leaders by country
countries = db["leaders"].find().distinct("country")

# Count the number of leaders for each country
leaders_by_country = {}
for country in countries:
    count = db["leaders"].count_documents({"country": country})
    leaders_by_country[country] = count


In [ ]:
# - Do the same by using a MongoDB [aggregation pipeline](https://www.mongodb.com/docs/manual/aggregation/)

temp = db["leaders"].aggregate([
    {"$group": {"_id": "$country", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
])

for country in temp:  
    print(country)

## Resources:
* [NoSQL Concepts (DataCamp)](https://www.datacamp.com/courses/nosql-concepts)
* [Introduction to MongoDB using Python (DataCamp)](https://www.datacamp.com/courses/introduction-to-using-mongodb-for-data-science-with-python)
* [Getting started with MongoDB](https://docs.mongodb.com/manual/tutorial/)
* [Python MongoDB Tutorial](https://www.mongodb.com/blog/post/getting-started-with-python-and-mongodb)
* [Introduction to MongoDB Learning Path](https://learn.mongodb.com/learning-paths/introduction-to-mongodb)
* [Build an App With Python, Flask, and MongoDB to Track UFOs](https://www.mongodb.com/developer/languages/python/flask-app-ufo-tracking/)